<a href="https://colab.research.google.com/github/prateekmanral011/Earthquake-forecasting/blob/main/mainshock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
##Declustering for magnitude Mw > 5
import pandas as pd
import numpy as np
from datetime import timedelta
from geopy.distance import geodesic

# === Load Earthquake Catalog ===
df = pd.read_csv('/content/homogenised_catalogue.csv')
df.columns = df.columns.str.strip()
df['date'] = pd.to_datetime(df['date'], format='mixed')
df = df.rename(columns={'Mw': 'Magnitude'})  # internal consistency
df = df.sort_values('date').reset_index(drop=True)

# === Gardner & Knopoff Time and Space Windows (days, km) ===
windows = [
    (5.0, 5.5, 223, 43.5),
    (5.5, 6.0, 400, 51),
    (6.0, 6.5, 680, 58),
    (6.5, 7.0, 883, 66),
    (7.0, 7.5, 938, 76),
    (7.5, 8.0, 973, 88),
    (8.0, 8.5, 1006, 102),
    (8.5, 9.2, 1065, 130)
]

def get_window(mag):
    for mag_min, mag_max, t_win, d_win in windows:
        if mag_min <= mag < mag_max:
            return t_win, d_win
    return 0, 0  # Events with mag < 5.0 will be ignored

# === Declustering Process ===
declustered_indices = []
used = np.zeros(len(df), dtype=bool)
aftershock_indices = []

for i, row in df.iterrows():
    if used[i]:
        continue

    mag = row['Magnitude']
    if mag < 5.0:
        continue  # Ignore events below threshold

    # Mark as mainshock
    declustered_indices.append(i)
    date = row['date']
    lat = row['latitude']
    lon = row['longitude']
    #id = row['id']

    t_win, d_win = get_window(mag)
    t_end = date + timedelta(days=t_win)

    for j in range(i + 1, len(df)):
        if df.loc[j, 'date'] > t_end:
            break
        if used[j]:
            continue

        # Compute distance and magnitude condition
        dist = geodesic((lat, lon), (df.loc[j, 'latitude'], df.loc[j, 'longitude'])).km
        if dist <= d_win and df.loc[j, 'Magnitude'] < mag:
            used[j] = True
            aftershock_indices.append(j)

# === Build Final DataFrames ===
mainshocks_df = df.loc[declustered_indices].reset_index(drop=True)
aftershocks_df = df.loc[aftershock_indices].reset_index(drop=True)

# === Format Columns for Output Consistency ===
mainshocks_df = mainshocks_df[['date', 'latitude', 'longitude', 'Magnitude', 'id']].rename(columns={'Magnitude': 'Mw'})
aftershocks_df = aftershocks_df[['date', 'latitude', 'longitude', 'Magnitude', 'id']].rename(columns={'Magnitude': 'Mw'})

# === Save to CSV Files ===
mainshocks_df.to_csv('mainshock_catalogue.csv', index=False)
aftershocks_df.to_csv('aftershocks_catalog_.csv', index=False)

# === Print Summary ===
print(f"✅ Total events in catalog: {len(df)}")
print(f"✅ Mainshocks identified: {len(mainshocks_df)}")
print(f"✅ Aftershocks identified: {len(aftershocks_df)}")

from google.colab import files

# Download both files
files.download('mainshock_catalogue.csv')
files.download('aftershocks_catalog_.csv')



✅ Total events in catalog: 7065
✅ Mainshocks identified: 4304
✅ Aftershocks identified: 2761


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>